In [1]:
import sqlite3
import pandas as pd
from src.models import *
import plotly.express as px
import numpy as np

### Get Data

In [2]:
with Session(engine) as session:
    query = (
        session
        .query(Event,TeamMemberEntry,Species)
        .join(Event.team_member_entries, isouter=True)
        .join(TeamMemberEntry.species, isouter=True)
    )
    print(query.statement)
    data = pd.read_sql(query.statement, engine)
data

SELECT event.no, event.playthrough_id_no, event.location_name, event.location_region, event.event_type_name, event.event_name, event.round, team_member_entry.no AS no_1, team_member_entry.team_member_playthrough_id_no, team_member_entry.team_member_slot, team_member_entry.event_no, team_member_entry.level, team_member_entry.species_name, species.name, species.dex_no, species.type1_name, species.type2_name 
FROM event LEFT OUTER JOIN team_member_entry ON event.no = team_member_entry.event_no LEFT OUTER JOIN species ON species.name = team_member_entry.species_name


,no,playthrough_id_no,location_name,location_region,event_type_name,event_name,round,no_1,team_member_playthrough_id_no,team_member_slot,event_no,level,species_name,name,dex_no,type1_name,type2_name
0,1,26852,Nuvema Town,Unova,Gift,Lillipup,0,1.0,26852,1.0,1.0,5.0,Lillipup,Lillipup,506.0,Normal,Normal
1,2,26852,Nuvema Town,Unova,Battle,PKMN Trainer Bianca,0,NaN,None,NaN,NaN,NaN,None,None,NaN,None,None
2,3,26852,Nuvema Town,Unova,Battle,PKMN Trainer Cheren,0,NaN,None,NaN,NaN,NaN,None,None,NaN,None,None
3,4,26852,Accumula Town,Unova,Battle,PKMN Trainer N,0,2.0,26852,1.0,4.0,7.0,None,None,NaN,None,None
4,5,26852,Route 2,Unova,Battle,Youngster Jimmy,0,3.0,26852,1.0,5.0,8.0,None,None,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,77,26852,Castelia City,Unova,Battle,Harlequin Kerry,0,NaN,None,NaN,NaN,NaN,None,None,NaN,None,None
83,78,26852,Castelia City,Unova,Battle,Harlequin Rick,0,NaN,None,NaN,NaN,NaN,None,None,NaN,None,None
84,79,26852,Castelia City,Unova,Battle,Harlequin Louis,0,47.0,26852,2.0,79.0,24.0,None,None,NaN,None,None
85,80,26852,Castelia City,Unova,Battle,Leader Burgh,0,49.0,26852,2.0,80.0,25.0,None,None,NaN,None,None


#### Level Data

In [3]:
df = (
    data
    .pivot_table(values='level',index='no',columns='team_member_slot')
    .reindex(np.arange(data['no'].min(),data['no'].max()+1))
    .reset_index()
    .ffill()
)
display(df)
px.line(df, x="no", y=[1,2,3]).update_layout(hovermode='x')

team_member_slot,no,1.0,2.0,3.0
0,1,5.0,NaN,NaN
1,2,5.0,NaN,NaN
2,3,5.0,NaN,NaN
3,4,7.0,NaN,NaN
4,5,8.0,NaN,NaN
...,...,...,...,...
75,76,24.0,23.0,23.0
76,77,24.0,23.0,23.0
77,78,24.0,23.0,23.0
78,79,24.0,24.0,23.0


## Levels Over Time

In [4]:
# px.line(data, x='')